## **Semantic segmentation**

Step:1
Importing all the necessary libraries to visualize the map and for model building.

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds
import IPython.display as disp

!pip install geemap
import geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.3 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none

Step:2
Authenticating the earth engine.

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=m_BT5-nBSIElHLAfeAiidnGF52N8z5lDfd6hLqNzAHA&tc=aBrScdR-BW4oEnvo-QJi7qP06gmTfqlFstZhQ5ejGp0&cc=OmOQYvCtOXXsWRgYitkk9-r9GHg4K9NPXyJMBgUnjX8

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VPY5xxJkFipk0d0Rf3S_72wiRh7r3WOThlRTwNUcSLWF2qGvl_HcX4

Successfully saved authorization token.


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import tensorflow as tf

#tf.enable_eager_execution()
print(tf.__version__)

2.12.0


Step:3
Initialize the earth engine and import the folium package for visualization.

In [ ]:
import ee
ee.Initialize()

# Test the earthengine command by getting help on upload.
!earthengine upload image -h

usage: earthengine upload image
       [-h]
       [--wait [WAIT]]
       [--force]
       [--asset_id ASSET_ID]
       [--last_band_alpha]
       [--nodata_value NODATA_VALUE]
       [--pyramiding_policy PYRAMIDING_POLICY]
       [--bands BANDS]
       [--crs CRS]
       [--manifest MANIFEST]
       [--property PROPERTY]
       [--time_start TIME_START]
       [--time_end TIME_END]
       [src_files ...]

Uploads an
image from
Cloud
Storage to
Earth
Engine. See
docs for
"asset set"
for
additional
details on
how to
specify
asset
metadata
properties.

positional arguments:
  src_files
    Cloud
    Storage
    URL(s) of
    the file(s)
    to upload.
    Must have
    the prefix
    'gs://'.

options:
  -h, --help
    show this
    help
    message and
    exit
  --wait [WAIT], -w [WAIT]
    Wait for
    the task to
    finish, or
    timeout
    after the
    specified
    number of
    seconds.
    Without
    this flag,
    the command
    just starts
    an export
    task in the
  

In [ ]:
import folium
print(folium.__version__)

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

0.14.0


Step:4
Collecting images from landsat 8 with corresponding bands and labels, appending them together.

In [ ]:
# Earth Engine username is used to import a classified image
# into the Earth Engine assets folder.
USER_NAME = 'ee-ssasikumar'

# I made a Cloud Storage bucket in google cloud into which training, testing and prediction datasets will be written.
OUTPUT_BUCKET = 'environment_project'

# Use Landsat 8 surface reflectance data for predictors.
L8SR = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
# Use these bands for prediction.
BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

# This is a training/testing dataset of points with known land cover labels.
LABEL_DATA = ee.FeatureCollection('projects/google/demo_landcover_labels')

# The labels, consecutive integer indices starting from zero, are stored in
# this property, set on each point.
LABEL = 'landcover'

# Number of label values, i.e. number of classes in the classification.
N_CLASSES = 3

# These names are used to specify properties in the export of training/testing data and to define the mapping between names and data
# when reading into TensorFlow datasets.
FEATURE_NAMES = list(BANDS)
FEATURE_NAMES.append(LABEL)

# File names for the training and testing datasets.  These TFRecord files
# will be exported from Earth Engine into the Cloud Storage bucket.
TRAIN_FILE_PREFIX = 'Training_demo'
TEST_FILE_PREFIX = 'Testing_demo'
file_extension = '.tfrecord.gz'
TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX + file_extension
TEST_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX + file_extension

# File name for the prediction (image) dataset.  The trained model will read
# this dataset and make predictions in each pixel.
IMAGE_FILE_PREFIX = 'Image_pixel_demo_'

# The output path for the classified image (i.e. predictions) TFRecord file.
OUTPUT_IMAGE_FILE = 'gs://' + OUTPUT_BUCKET + '/Classified_pixel_demo.TFRecord'

# Export imagery in this region.
EXPORT_REGION = ee.Geometry.Rectangle([-122.7, 37.3, -121.8, 38.00])#roi

# Path to the Earth Engine asset for importing  the classified image from the TFRecord file in Cloud Storage.
OUTPUT_ASSET_ID = 'projects/ee-ssasikumar/assets/Classified_pixel_demo'

Step:5
Create the mask function to remove the cloud and visualize it using folium on the map.

In [ ]:
# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(BANDS).divide(10000)

# The image input data is a 2018 cloud-masked median composite.
image = L8SR.filterDate('2018-01-01', '2018-12-31').map(maskL8sr).median()

# Using folium to visualize the imagery.
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[9.714855444616322, 76.32380968048625])

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Step:6
Sample the regions and split it into testing and training batches.

In [ ]:
# Sample the image at the points and add a random column.
sample = image.sampleRegions(
  collection=LABEL_DATA, properties=[LABEL], scale=30).randomColumn()

# Partition the sample approximately 70-30.
training = sample.filter(ee.Filter.lt('random', 0.7))
testing = sample.filter(ee.Filter.gte('random', 0.7))

from pprint import pprint

# Print the first couple points to verify.
pprint({'training': training.first().getInfo()})
pprint({'testing': testing.first().getInfo()})

{'training': {'geometry': None,
              'id': '000066e7d9bc84b3f95d_0',
              'properties': {'B2': 0.049150001257658005,
                             'B3': 0.06965000182390213,
                             'B4': 0.08974999934434891,
                             'B5': 0.1729000061750412,
                             'B6': 0.2125999927520752,
                             'B7': 0.15150000154972076,
                             'landcover': 1,
                             'random': 0.5484198857675888},
              'type': 'Feature'}}
{'testing': {'geometry': None,
             'id': '00009f65e3c9ae02b84e_0',
             'properties': {'B2': 0.05220000073313713,
                            'B3': 0.062049999833106995,
                            'B4': 0.03660000115633011,
                            'B5': 0.01140000019222498,
                            'B6': 0.006800000090152025,
                            'B7': 0.005249999929219484,
                            'landcover'

In [ ]:
# Checking the existance of the output bucket.
print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + OUTPUT_BUCKET)
    else 'Can not find output Cloud Storage bucket.')


Found Cloud Storage bucket.


Step:7
Creating the training and testing task and export it to cloud storage.

In [ ]:
# Create the tasks.
training_task = ee.batch.Export.table.toCloudStorage(
  collection=training,
  description='Training Export',
  fileNamePrefix=TRAIN_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

testing_task = ee.batch.Export.table.toCloudStorage(
  collection=testing,
  description='Testing Export',
  fileNamePrefix=TEST_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

In [ ]:
# Start the tasks.
training_task.start()
testing_task.start()

In [ ]:
# Print all tasks.
pprint(ee.batch.Task.list())


[<Task 6JXILDICWEPVXDCOYSNPTTIG EXPORT_FEATURES: Testing Export (READY)>,
 <Task RQALUQND3TA7FSJCVO3USUMY EXPORT_FEATURES: Training Export (READY)>,
 <Task R3MQFWQSCIFOWK7PFHRB5V3X EXPORT_IMAGE: Image Export (FAILED)>,
 <Task 6ODV7YFZFA7RBIT4E6HJUJ2Q EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task O65CGNYRYNUZIWCWGNU37MLJ EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task 5RLDGMAGFB5TN3IELTLJT4RK INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/ee-ssasikumar/Classified_pixel_demo" (FAILED)>,
 <Task ZR2X6J5QLX3HF4UVNEI22P54 INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/ee-ssasikumar/Classified_pixel_demo" (FAILED)>,
 <Task TJBCMZ4M2JTJPITDYNZTY75Y EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task 4D3C4Z5RDG5EZQMXII3MTQQN EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task RGWIGAO2OEZRLV4MCQEHJ2ZX EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task OK5GMIJ6ULXWVM4AM6A2VQGT INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/asset

In [ ]:
# Make sure you can see the output bucket.  You must have write access.
print('Found training file.' if tf.io.gfile.exists(TRAIN_FILE_PATH)
    else 'No training file found.')
print('Found testing file.' if tf.io.gfile.exists(TEST_FILE_PATH)
    else 'No testing file found.')

No training file found.
No testing file found.


Step:8
Export these images to the cloud storage, on which prediction will be carried out.





In [ ]:
# Specify patch and file dimensions.
image_export_options = {
  'patchDimensions': [256, 256],
  'maxFileSize': 104857600,
  'compressed': True
}

# Setup the task.
image_task = ee.batch.Export.image.toCloudStorage(
  image=image,
  description='Image Export',
  fileNamePrefix=IMAGE_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  scale=30,
  fileFormat='TFRecord',
  region=EXPORT_REGION.toGeoJSON()['coordinates'],#Any cordinates from geojson can be inserted here as roi
  formatOptions=image_export_options,
)

In [ ]:
image_task.start()

In [ ]:
pprint(ee.batch.Task.list())

[<Task RZH7BYBAP6GB7EQRLYOT4X2I EXPORT_IMAGE: Image Export (READY)>,
 <Task 6JXILDICWEPVXDCOYSNPTTIG EXPORT_FEATURES: Testing Export (READY)>,
 <Task RQALUQND3TA7FSJCVO3USUMY EXPORT_FEATURES: Training Export (READY)>,
 <Task R3MQFWQSCIFOWK7PFHRB5V3X EXPORT_IMAGE: Image Export (FAILED)>,
 <Task 6ODV7YFZFA7RBIT4E6HJUJ2Q EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task O65CGNYRYNUZIWCWGNU37MLJ EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task 5RLDGMAGFB5TN3IELTLJT4RK INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/ee-ssasikumar/Classified_pixel_demo" (FAILED)>,
 <Task ZR2X6J5QLX3HF4UVNEI22P54 INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/ee-ssasikumar/Classified_pixel_demo" (FAILED)>,
 <Task TJBCMZ4M2JTJPITDYNZTY75Y EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task 4D3C4Z5RDG5EZQMXII3MTQQN EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task RGWIGAO2OEZRLV4MCQEHJ2ZX EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task OK5GMIJ6ULXWVM4AM6

In [ ]:
import time

while image_task.active():
  print('Polling for task (id: {}).'.format(image_task.id))
  time.sleep(30)
print('Done with image export.')

Polling for task (id: RZH7BYBAP6GB7EQRLYOT4X2I).
Polling for task (id: RZH7BYBAP6GB7EQRLYOT4X2I).
Polling for task (id: RZH7BYBAP6GB7EQRLYOT4X2I).
Polling for task (id: RZH7BYBAP6GB7EQRLYOT4X2I).
Polling for task (id: RZH7BYBAP6GB7EQRLYOT4X2I).
Polling for task (id: RZH7BYBAP6GB7EQRLYOT4X2I).
Polling for task (id: RZH7BYBAP6GB7EQRLYOT4X2I).
Polling for task (id: RZH7BYBAP6GB7EQRLYOT4X2I).
Polling for task (id: RZH7BYBAP6GB7EQRLYOT4X2I).
Polling for task (id: RZH7BYBAP6GB7EQRLYOT4X2I).
Polling for task (id: RZH7BYBAP6GB7EQRLYOT4X2I).
Done with image export.


Step:9
Make the training data into TFR format for reading.


In [ ]:
# Create a dataset from the TFRecord file in Cloud Storage.
train_dataset = tf.data.TFRecordDataset(TRAIN_FILE_PATH, compression_type='GZIP')
# Print the first record to check.
print(iter(train_dataset).next())

tf.Tensor(b'\nw\n\x0e\n\x02B2\x12\x08\x12\x06\n\x04\x83QI=\n\x0e\n\x02B3\x12\x08\x12\x06\n\x04\xa9\xa4\x8e=\n\x0e\n\x02B4\x12\x08\x12\x06\n\x04\xd9\xce\xb7=\n\x0e\n\x02B5\x12\x08\x12\x06\n\x04\xb3\x0c1>\n\x0e\n\x02B6\x12\x08\x12\x06\n\x04\xd0\xb3Y>\n\x0e\n\x02B7\x12\x08\x12\x06\n\x04\xd1"\x1b>\n\x15\n\tlandcover\x12\x08\x12\x06\n\x04\x00\x00\x80?', shape=(), dtype=string)


In [ ]:
# List of fixed-length features, all of which are float32.
columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in FEATURE_NAMES
]

# Dictionary with names as keys, features as values.
features_dict = dict(zip(FEATURE_NAMES, columns))

pprint(features_dict)

{'B2': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B3': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B4': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B5': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B6': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B7': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'landcover': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None)}


Step:10
Now we need to make a parsing function for the data in the TFRecord files. The data comes in flattened 2D arrays per record and we want to use the first part of the array for input to the model and the last element of the array as the class label. The parsing function reads data from a serialized Example proto into a dictionary in which the keys are the feature names and the values are the tensors storing the value of the features for that example.

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by featuresDict.

  Args:
    example_proto: a serialized Example.

  Returns:
    A tuple of the predictors dictionary and the label, cast to an `int32`.
  """
  parsed_features = tf.io.parse_single_example(example_proto, features_dict)
  labels = parsed_features.pop(LABEL)
  return parsed_features, tf.cast(labels, tf.int32)

# Map the function over the dataset.
parsed_dataset = train_dataset.map(parse_tfrecord, num_parallel_calls=5)

# Print the first parsed record to check.
pprint(iter(parsed_dataset).next())

({'B2': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.04915], dtype=float32)>,
  'B3': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.06965], dtype=float32)>,
  'B4': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.08975], dtype=float32)>,
  'B5': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1729], dtype=float32)>,
  'B6': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.2126], dtype=float32)>,
  'B7': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1515], dtype=float32)>},
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>)


step:11
Another thing we might want to do as part of the input process is to create new features, for example NDVI, a vegetation index computed from reflectance in two spectral bands B5 and B4.This is the helper functions for that.

In [ ]:
def normalized_difference(a, b):
  """Compute normalized difference of two inputs.

  Compute (a - b) / (a + b).  If the denomenator is zero, add a small delta.

  Args:
    a: an input tensor with shape=[1]
    b: an input tensor with shape=[1]

  Returns:
    The normalized difference as a tensor.
  """
  nd = (a - b) / (a + b)
  nd_inf = (a - b) / (a + b + 0.000001)
  return tf.where(tf.math.is_finite(nd), nd, nd_inf)

def add_NDVI(features, label):
  """Add NDVI to the dataset.
  Args:
    features: a dictionary of input tensors keyed by feature name.
    label: the target label

  Returns:
    A tuple of the input dictionary with an NDVI tensor added and the label.
  """
  features['NDVI'] = normalized_difference(features['B5'], features['B4'])
  return features, label

Step:12
Building the model using keras and fit it with training data.

In [ ]:
from tensorflow import keras

# Add NDVI.
input_dataset = parsed_dataset.map(add_NDVI)

# Keras requires inputs as a tuple.  Note that the inputs must be in the
# right shape.  Also note that to use the categorical_crossentropy loss,
# the label needs to be turned into a one-hot vector.
def to_tuple(inputs, label):
  return (tf.transpose(list(inputs.values())),
          tf.one_hot(indices=label, depth=N_CLASSES))

# Map the to_tuple function, shuffle and batch.
input_dataset = input_dataset.map(to_tuple).batch(8)

# Define the layers in the model.
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(N_CLASSES, activation=tf.nn.softmax)
])

# Compile the model with the specified loss function.
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fit the model to the training data.
model.fit(x=input_dataset, epochs=10)

Epoch 1/10
9/9 [==============================] - 1s 13ms/step - loss: 1.0401 - accuracy: 0.6286
Epoch 2/10
9/9 [==============================] - 0s 9ms/step - loss: 1.0107 - accuracy: 0.8571
Epoch 3/10
9/9 [==============================] - 0s 6ms/step - loss: 0.9654 - accuracy: 0.9143
Epoch 4/10
9/9 [==============================] - 0s 5ms/step - loss: 0.9441 - accuracy: 0.9143
Epoch 5/10
9/9 [==============================] - 0s 5ms/step - loss: 0.9088 - accuracy: 0.9714
Epoch 6/10
9/9 [==============================] - 0s 5ms/step - loss: 0.8724 - accuracy: 0.9429
Epoch 7/10
9/9 [==============================] - 0s 4ms/step - loss: 0.8293 - accuracy: 0.9429
Epoch 8/10
9/9 [==============================] - 0s 3ms/step - loss: 0.8068 - accuracy: 0.9429
Epoch 9/10
9/9 [==============================] - 0s 3ms/step - loss: 0.7705 - accuracy: 0.9571
Epoch 10/10
9/9 [==============================] - 0s 3ms/step - loss: 0.7385 - accuracy: 0.9429


step:13
Evaluate the model

In [ ]:
test_dataset = (
  tf.data.TFRecordDataset(TEST_FILE_PATH, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .map(add_NDVI)
    .map(to_tuple)
    .batch(1))

model.evaluate(test_dataset)

28/28 [==============================] - 0s 2ms/step - loss: 0.7772 - accuracy: 0.8214


[0.777238667011261, 0.8214285969734192]

In [ ]:
# Get a list of all the files in the output bucket.
files_list = !gsutil ls 'gs://'{OUTPUT_BUCKET}
# Get only the files generated by the image export.
exported_files_list = [s for s in files_list if IMAGE_FILE_PREFIX in s]

# Get the list of image files and the JSON mixer file.
image_files_list = []
json_file = None
for f in exported_files_list:
  if f.endswith('.tfrecord.gz'):
    image_files_list.append(f)
  elif f.endswith('.json'):
    json_file = f

# Make sure the files are in the right order.
image_files_list.sort()

pprint(image_files_list)
print(json_file)

['gs://environment_project/Image_pixel_demo_00000.tfrecord.gz',
 'gs://environment_project/Image_pixel_demo_00001.tfrecord.gz']
gs://environment_project/Image_pixel_demo_mixer.json


In [ ]:
import json

# Load the contents of the mixer file to a JSON object.
json_text = !gsutil cat {json_file}
# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(json_text.nlstr)
pprint(mixer)

{'patchDimensions': [256, 256],
 'patchesPerRow': 13,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            -122.70007617412975,
                                            0.0,
                                            -0.00026949458523585647,
                                            38.00089247493765]},
                'crs': 'EPSG:4326'},
 'totalPatches': 130}


In [ ]:
# Get relevant info from the JSON mixer file.
patch_width = mixer['patchDimensions'][0]
patch_height = mixer['patchDimensions'][1]
patches = mixer['totalPatches']
patch_dimensions_flat = [patch_width * patch_height, 1]

# Note that the tensors are in the shape of a patch, one patch for each band.
image_columns = [
  tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32)
    for k in BANDS
]

# Parsing dictionary.
image_features_dict = dict(zip(BANDS, image_columns))

# Note that you can make one dataset from many files by specifying a list.
image_dataset = tf.data.TFRecordDataset(image_files_list, compression_type='GZIP')

# Parsing function.
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, image_features_dict)

# Parse the data into tensors, one long tensor per patch.
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

# Break our long tensors into many little ones.
image_dataset = image_dataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)

# Add additional features (NDVI).
image_dataset = image_dataset.map(
  # Add NDVI to a feature that doesn't have a label.
  lambda features: add_NDVI(features, None)[0]
)

# Turn the dictionary in each record into a tuple without a label.
image_dataset = image_dataset.map(
  lambda data_dict: (tf.transpose(list(data_dict.values())), )
)

# Turn each patch into a batch.
image_dataset = image_dataset.batch(patch_width * patch_height)

Step:14
Do the prediction and write to cloud in TFRecord format.

In [ ]:
# Run prediction in batches, with as many steps as there are patches.
predictions = model.predict(image_dataset, steps=patches, verbose=1)

# Note that the predictions come as a numpy array.  Check the first one.
print(predictions[0])

130/130 [==============================] - 512s 4s/step
[[0.24734567 0.6163701  0.13628428]]


In [ ]:
print('Writing to file ' + OUTPUT_IMAGE_FILE)

Writing to file gs://environment_project/Classified_pixel_demo.TFRecord


In [ ]:
# Instantiate the writer.
writer = tf.io.TFRecordWriter(OUTPUT_IMAGE_FILE)

# Every patch-worth of predictions we'll dump an example into the output file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here will also be in the right order.

patch = [[], [], [], []]
cur_patch = 1
for prediction in predictions:
  patch[0].append(tf.argmax(prediction, 1))
  patch[1].append(prediction[0][0])
  patch[2].append(prediction[0][1])
  patch[3].append(prediction[0][2])
  # Once we've seen a patches-worth of class_ids...
  if (len(patch[0]) == patch_width * patch_height):
    print('Done with patch ' + str(cur_patch) + ' of ' + str(patches) + '...')
    # Create an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[0])),
          'bareProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[1])),
          'vegProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[2])),
          'waterProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[3])),
        }
      )
    )
    # Write the example to the file and clear our patch array so it's ready for
    # another batch of   class ids
    writer.write(example.SerializeToString())
    patch = [[], [], [], []]
    cur_patch += 1

writer.close()

Done with patch 1 of 130...
Done with patch 2 of 130...
Done with patch 3 of 130...
Done with patch 4 of 130...
Done with patch 5 of 130...
Done with patch 6 of 130...
Done with patch 7 of 130...
Done with patch 8 of 130...
Done with patch 9 of 130...
Done with patch 10 of 130...
Done with patch 11 of 130...
Done with patch 12 of 130...
Done with patch 13 of 130...
Done with patch 14 of 130...
Done with patch 15 of 130...
Done with patch 16 of 130...
Done with patch 17 of 130...
Done with patch 18 of 130...
Done with patch 19 of 130...
Done with patch 20 of 130...
Done with patch 21 of 130...
Done with patch 22 of 130...
Done with patch 23 of 130...
Done with patch 24 of 130...
Done with patch 25 of 130...
Done with patch 26 of 130...
Done with patch 27 of 130...
Done with patch 28 of 130...
Done with patch 29 of 130...
Done with patch 30 of 130...
Done with patch 31 of 130...
Done with patch 32 of 130...
Done with patch 33 of 130...
Done with patch 34 of 130...
Done with patch 35 of 1

In [ ]:
!gsutil ls -l {OUTPUT_IMAGE_FILE}

 136331260  2023-05-24T13:17:36Z  gs://environment_project/Classified_pixel_demo.TFRecord
TOTAL: 1 objects, 136331260 bytes (130.02 MiB)


In [ ]:
print('Uploading to ' + OUTPUT_ASSET_ID)

Uploading to projects/ee-ssasikumar/assets/Classified_pixel_demo


Step:15
Uplload the predicted tiles to earth engine assets and ovelay on the map for visualization.

In [ ]:
# Start the upload.
!earthengine upload image --asset_id={OUTPUT_ASSET_ID} --pyramiding_policy=mode {OUTPUT_IMAGE_FILE} {json_file}

Started upload task with ID: UT5AVQK3EUOWV47KS5PNRK3U


In [ ]:
ee.batch.Task.list()

[<Task J6RZEJ7XPP3GZCGCVVMIS4K4 INGEST_IMAGE: Ingest image: "projects/ee-ssasikumar/assets/Classified_pixel_demo" (READY)>,
 <Task 56SMSYDH7NUMQIUOJRM5FXFZ INGEST_IMAGE: Ingest image: "projects/ee-ssasikumar/assets/Image_pixel_demo_mixer" (RUNNING)>,
 <Task I6KQEJEWDJTHPIDXSD6TBK5E INGEST_IMAGE: Ingest image: "projects/ee-ssasikumar/assets/Image_pixel_demo_mixer" (RUNNING)>,
 <Task WLXZULEG2IJPOERRM464C3TV INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/ee-ssasikumar/Classified_pixel_demo" (FAILED)>,
 <Task RZH7BYBAP6GB7EQRLYOT4X2I EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task 6JXILDICWEPVXDCOYSNPTTIG EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task RQALUQND3TA7FSJCVO3USUMY EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task R3MQFWQSCIFOWK7PFHRB5V3X EXPORT_IMAGE: Image Export (FAILED)>,
 <Task 6ODV7YFZFA7RBIT4E6HJUJ2Q EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task O65CGNYRYNUZIWCWGNU37MLJ EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task 5RLDGMAG

In [ ]:
predictions_image = ee.Image(OUTPUT_ASSET_ID)

prediction_vis = {
  'bands': 'prediction',
  'min': 0,
  'max': 2,
  'palette': ['red', 'green', 'blue']
}
probability_vis = {'bands': ['bareProb', 'vegProb', 'waterProb'], 'max': 0.5}

prediction_map_id = predictions_image.getMapId(prediction_vis)
probability_map_id = predictions_image.getMapId(probability_vis)

map = folium.Map(location=[38., -122.5])
folium.TileLayer(
  tiles=prediction_map_id['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='prediction',
).add_to(map)
folium.TileLayer(
  tiles=probability_map_id['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='probability',
).add_to(map)
map.add_child(folium.LayerControl())
map